In [ ]:
### OLD

import json
import time
import math 

import pandas as pd
import requests
import unidecode
from bs4 import BeautifulSoup

from 

In [ ]:
# Get database credentials

f = open("database.txt", "r")
lines = f.readlines()

username = lines[0].replace("\n", "")
password = lines[1].replace("\n", "")
host = lines[2].replace("\n", "")
port = lines[3].replace("\n", "")
table = lines[4].replace("\n", "")
f.close()

# print(
#     f"username: {username}, password: {password}, host: {host}, port: {port}, table: {table}"
# )

In [ ]:
offer_url_1 = "https://www.otodom.pl/pl/oferta/2-pok-55-m2-z-widokiem-na-ogrod-krasinskich-ID4lszi"
offer_url_2 = (
    "https://www.otodom.pl/pl/oferta/widokowe-3-pokoje-przy-metrze-wawrzyszew-ID4lxZ0"
)
listing_url_1 = "https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?distanceRadius=0&locations=%5Bcities_6-26%5D&viewType=listing"

In [ ]:
# Pobieranie parametrów oferty
def get_offer_params(offer_url):
    """
    Docstring
    """
    r = requests.get(offer_url)
    soup = BeautifulSoup(r.content, "html.parser")

    offer_params = soup.find_all("div", {"class": "css-1wi2w6s enb64yk4"})

    results = dict()

    price = soup.find("strong", {"aria-label": "Cena"})
    results["price"] = unidecode.unidecode(price.text)

    price_m2 = soup.find("div", {"aria-label": "Cena za metr kwadratowy"})
    results["price_m2"] = unidecode.unidecode(price_m2.text)

    address = soup.find("a", {"aria-label": "Adres"})
    results["address"] = unidecode.unidecode(address.text)

    for line in offer_params:
        k = unidecode.unidecode(
            str(line.parent.parent["aria-label"])
            .replace(" / ", "_")
            .replace(" ", "_")
            .lower()
        )
        v = unidecode.unidecode(str(line.text).strip())
        results[k] = v

    return results

In [ ]:
# Pobieranie linków z listingu
def get_offers_urls_from_single_page(listing_url):
    """
    Docstring
    """

    r = requests.get(listing_url)
    soup = BeautifulSoup(r.content, "html.parser")

    urls_json = soup.find("script", {"id": "__NEXT_DATA__"}).get_text()

    json_dict = json.loads(urls_json)
    offers = json_dict["props"]["pageProps"]["schemaMarkupData"]["@graph"][2]["offers"][
        "offers"
    ]

    offers_list = []
    for offer in offers:
        offers_list.append({"name": offer["name"], "url": offer["url"]})

    return offers_list

In [ ]:
# Pobieranie linków z listingu
def get_offers_urls_from_all_pages(listing_url):
    """
    Docstring
    """

    r = requests.get(listing_url)
    soup = BeautifulSoup(r.content, "html.parser")

    offers_number = int(soup.find("span", {"class": "css-19fwpg e17mqyjp2"}).get_text())
    offers_per_page = 36
    pages_number = math.ceil(offers_number/offers_per_page)

    offers_list = []
    for page_num in range(1, pages_number):
        time.sleep(3)
        listing_page = f"{listing_url}&page={page_num}"
        print(listing_page)

        offers_list.append(get_offers_urls_from_single_page(listing_page))

    return offers_list

In [ ]:
# Pobieranie ofert z linków z listingu
def create_offers_table(offers_list):
    """
    Docstring
    """
    results = list()
    for offer in offers_list:
        enriched_offer = dict()

        enriched_offer["offer_name"] = offer["name"]
        enriched_offer["offer_url"] = offer["url"]

        enriched_offer = {**enriched_offer, **get_offer_params(offer["url"])}
        results.append(enriched_offer)

    return results

In [ ]:
offers_urls = get_offers_urls_from_single_page("https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?distanceRadius=0&locations=%5Bcities_6-26%5D&viewType=listing&page=2&page=3")

In [ ]:
offers_urls

In [ ]:
# urls_json = soup.find("script", {"id": "__NEXT_DATA__"}).get_text()

# json_dict = json.loads(urls_json)
# offers = json_dict["props"]["pageProps"]["schemaMarkupData"]["@graph"][2]["offers"][
#     "offers"
# ]

# offers_list = []
# for offer in offers:
#     offers_list.append({"name": offer["name"], "url": offer["url"]})

In [ ]:
r = requests.get("https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?distanceRadius=0&locations=%5Bcities_6-26%5D&viewType=listing")
soup = BeautifulSoup(r.content, "html.parser")

In [ ]:
# urls_json = soup.find("script", {"id": "__NEXT_DATA__"}).get_text()

In [ ]:
# json_dict = json.loads(urls_json)

In [ ]:
# offers = json_dict["props"]["pageProps"]["schemaMarkupData"]["@graph"][2]["offers"][
#     "offers"
# ]

In [ ]:
offers = soup.find_all("li", {"data-cy": "listing-item"})

In [ ]:
len(offers)

In [ ]:
for offer in offers:
    a = offer.find("a", {"data-cy": "listing-item-link"})
    print(a["href"])

In [ ]:
with open("Output.txt", "w") as text_file:
    text_file.write(str(json_dict["props"]["pageProps"]))

In [ ]:
# str(json_dict["props"]["pageProps"])

In [ ]:
offers_listing = create_offers_table(get_offers_urls(listing_url_1))

In [ ]:
df = pd.DataFrame(offers_listing)

In [ ]:
# df

In [ ]:
for i in range(1, 10):
    time.sleep(1)
    print(f"{listing_url_1}&page={i}")

In [ ]:
r = requests.get(f"{listing_url_1}&page=1")
soup = BeautifulSoup(r.content, "html.parser")

In [ ]:
# print(soup.prettify())

In [ ]:
offers_number = soup.find("span", {"class": "css-19fwpg e17mqyjp2"})

In [ ]:
int(offers_number.get_text())

In [ ]:
offers_per_page = soup.find("input", {"name": "entriesPerPage"}).get_text()

In [ ]:
math.ceil(6.1)

In [ ]:
offers_per_page['value']

In [ ]:
import datetime

In [ ]:
df.insert(loc=0, column="create_timestamp", value=datetime.datetime.now())

In [ ]:
df

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{table}")
df.to_sql("otodom_offers_1", engine, if_exists="append", index=False)

In [ ]:
round

In [ ]:
df.to_csv(f"results_{int(time.time())}.csv", index=False)